In this notebook, we will see how we can make use nnUnet to make masks for this competition. 

What is [nnUnet](https://github.com/MIC-DKFZ/nnUNet)?
>nnU-Net is the first segmentation method that is designed to deal with the dataset diversity found in the domain (medical). It condenses and automates the key decisions for designing a successful segmentation pipeline for any given dataset. 
nnUnet was published in [Nature](https://www.nature.com/articles/s41592-020-01008-z)

Abstract from nature:
>Biomedical imaging is a driver of scientific discovery and a core component of medical care and is being stimulated by the field of deep learning. While semantic segmentation algorithms enable image analysis and quantification in many applications, the design of respective specialized solutions is non-trivial and highly dependent on dataset properties and hardware conditions. We developed nnU-Net, a deep learning-based segmentation method that automatically configures itself, including preprocessing, network architecture, training and post-processing for any new task. The key design choices in this process are modeled as a set of fixed parameters, interdependent rules and empirical decisions. Without manual intervention, nnU-Net surpasses most existing approaches, including highly specialized solutions on 23 public datasets used in international biomedical segmentation competitions. We make nnU-Net publicly available as an out-of-the-box tool, rendering state-of-the-art segmentation accessible to a broad audience by requiring neither expert knowledge nor computing resources beyond standard network training.

nnUnet lead the [Medical Decathlon competition](http://medicaldecathlon.com/results/).  

We will make use of nnUnet's Task01-Brain Tumouur Segmentation pretrained model to generate masks. 

We will follow the following steps
1. Select 5 random patients
2. Resample the images using SimpleITK and SIR24 dataset - I have a [notebook](https://www.kaggle.com/marshath/btrg-dicom-to-nifti-using-sri24-to-resample) completely for this
3. Git clone and set up nnUnet - This [notebook](https://github.com/prateekgupta891/nnUNet/blob/master/nnunetmec2020.ipynb) here was my light
4. Prepare the data as per nnUnet's requirement
5. Make masks!

# 0. Imports and paths

In [ ]:
import os
from fastai.vision.all import *
import SimpleITK as sitk
import shutil

# 1. Convert Dicom to NIFTI images

In [ ]:
def dicom2nifti(image_dir, out_dir, save=True):
    "given a dicom directory, loads them into single file and can save it as .nii file"
    reader = sitk.ImageSeriesReader()
    reader.LoadPrivateTagsOn()
    filenamesDICOM = reader.GetGDCMSeriesFileNames(str(image_dir))
    reader.SetFileNames(filenamesDICOM)
    img = reader.Execute()
    img = sitk.Cast(img, sitk.sitkFloat32)
    
    if save:
        sitk.WriteImage(img, f'{out_dir}/{image_dir.parent.name}.nii')
    else:
        return img

# 2. Resample the images using SimpleITK and SIR24 dataset

As we can see, for orientations are different across the different modalities. Let's resample using SRI24 as the template. 

In [ ]:
def resample_nifti(image_dir, ref_image, fn, nifti_img=None, save=True):
    "resample using a reference image"
    
    if not nifti_img:
        image = sitk.ReadImage(str(image_dir), sitk.sitkFloat32)
    else:
        image = nifti_img
    
    initial_transform = sitk.CenteredTransformInitializer(ref_image, 
                                                          image, 
                                                          sitk.Euler3DTransform(), 
                                                          sitk.CenteredTransformInitializerFilter.GEOMETRY)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ref_image)
    resampler.SetInterpolator(sitk.sitkLinear)
    resampler.SetTransform(initial_transform)
    resampler.SetOutputSpacing(ref_image.GetSpacing())
    resampler.SetSize((ref_image.GetSize()))
    resampler.SetOutputDirection(ref_image.GetDirection())
    resampler.SetOutputOrigin(ref_image.GetOrigin())
    resampler.SetDefaultPixelValue(image.GetPixelIDValue())
    resamped_image = resampler.Execute(image)
    
    if save:
        print(f"Saving NIFTI image at {fn}")
        sitk.WriteImage(resamped_image, fn)

    return resamped_image

In [ ]:
ref_image = sitk.ReadImage('../input/sri24-dataset/sri24/spgr.nii', sitk.sitkFloat32)

In [ ]:
imgd = sitk.GetArrayFromImage(ref_image)
print(imgd.shape)
plt.imshow(imgd[100], cmap='gray')

Looks like we have resampled them. If you noticed, we added four digits to the file names. This is because nnUnet (Task01 - Brain Tumour) requires all four modalities 

`0000 - flair, 0001 - T1w, 0002 - T1wce, 0003 - T2w`

# 3. Git clone and set up nnUnet

`labels: {'0': 'background',  '1': 'edema', '2': 'non-enhancing tumor', '3': 'enhancing tumour'}` - following is the labels 

In [ ]:
def make_if_dont_exist(folder_path,overwrite=False):
    """
    creates a folder if it does not exists
    input: 
    folder_path : relative path of the folder which needs to be created
    over_write :(default: False) if True overwrite the existing folder 
    """
    if os.path.exists(folder_path):
        
        if not overwrite:
            print(f'{folder_path} exists.')
        else:
            print(f"{folder_path} overwritten")
            shutil.rmtree(folder_path)
            os.makedirs(folder_path)

    else:
        os.makedirs(folder_path)
        print(f"{folder_path} created!")

In [ ]:
make_if_dont_exist('input/nnunet')
!cp -rf ../input/nnunet input/nnunet
respository_dir = 'input/nnunet/nnunet'
os.chdir(respository_dir)

!pip install -e .

os.chdir("/kaggle/working")

In [ ]:
# os.chdir("/kaggle/working")

In [ ]:
# import os
# base_dir = '/kaggle/working/'
# os.chdir(base_dir)

In [ ]:
# !git clone https://github.com/MIC-DKFZ/nnUNet.git 
# !git clone https://github.com/NVIDIA/apex

In [ ]:
# respository_dir = os.path.join(base_dir,'nnUNet')
# os.chdir(respository_dir)

# !pip install -e .
# #(optional installation)
# !pip install --upgrade git+https://github.com/nanohanno/hiddenlayer.git@bugfix/get_trace_graph#egg=hiddenlayer

# os.chdir(base_dir)

**Note** Restart the notebook

In [ ]:
# import IPython
# IPython.Application.instance().kernel.do_shutdown(True)

In [ ]:
import os
from fastai.vision.all import *
import SimpleITK as sitk
import shutil

base_dir = '/kaggle/working/'
os.chdir(base_dir)

## Dataset Folder Structure

In [ ]:
# def make_if_dont_exist(folder_path,overwrite=False):
#     """
#     creates a folder if it does not exists
#     input: 
#     folder_path : relative path of the folder which needs to be created
#     over_write :(default: False) if True overwrite the existing folder 
#     """
#     if os.path.exists(folder_path):
        
#         if not overwrite:
#             print(f'{folder_path} exists.')
#         else:
#             print(f"{folder_path} overwritten")
#             shutil.rmtree(folder_path)
#             os.makedirs(folder_path)

#     else:
#         os.makedirs(folder_path)
#         print(f"{folder_path} created!")

In [ ]:
# !ls nunet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data

In [ ]:
# task_name = 'Task001_BrainTumour' #change here for different task name
# nnunet_dir = "nnunet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data"
# task_folder_name = os.path.join(nnunet_dir,task_name)
# train_image_dir = os.path.join(task_folder_name,'imagesTr')
# train_label_dir = os.path.join(task_folder_name,'labelsTr')
# test_dir = os.path.join(task_folder_name,'imagesTs')
# main_dir = os.path.join(base_dir,'nnUNet/nnunet')

In [ ]:
# make_if_dont_exist(task_folder_name,overwrite = False)
# make_if_dont_exist(train_image_dir)
# make_if_dont_exist(train_label_dir)
# make_if_dont_exist(test_dir,overwrite= False)
# make_if_dont_exist(os.path.join(main_dir,'nnunet_trained_models'))

In [ ]:
!ls /kaggle/working/input/nnunet-pretrained-brain-tumor-segmentation-network

In [ ]:
nnunet_dir = "/kaggle/working/input/nnunet/nnunet"
intermediate_dir = "/kaggle/working/input/nnUNet_raw_data_base"

task_name = 'Task001_BrainTumour' #change here for different task name
task_folder_name = os.path.join(intermediate_dir, 'nnUNet_raw_data', task_name)
test_dir = os.path.join(task_folder_name,'imagesTs')


output_dir = "/kaggle/working/output/segmented"
pretrained_model_dir = "/kaggle/working/input/nnunet-pretrained-brain-tumor-segmentation-network/nnUNet"

make_if_dont_exist(intermediate_dir, overwrite = False)
make_if_dont_exist(output_dir, overwrite = False)
make_if_dont_exist(test_dir, overwrite = False)
make_if_dont_exist(pretrained_model_dir, overwrite = False)

In [ ]:
pretrained_model_dir_new = "/kaggle/working/input/nnunet-pretrained-brain-tumor-segmentation-network"
!cp -rf ../input/nnunet-pretrained-brain-tumor-segmentation-network/* input/nnunet-pretrained-brain-tumor-segmentation-network/nnUNet

In [ ]:
# !ls input/nnunet-pretrained-brain-tumor-segmentation-network/nnUNet

In [ ]:
# !ls input/nnunet-pretrained-brain-tumor-segmentation-network/nnUNet

## Environment Veriables

In [ ]:
# os.environ['nnUNet_raw_data_base'] = os.path.join(main_dir,'nnUNet_raw_data_base')
# os.environ['nnUNet_preprocessed'] = os.path.join(main_dir,'preprocessed')
# os.environ['RESULTS_FOLDER'] = os.path.join(main_dir,'nnUNet_trained_models')

In [ ]:
os.environ['nnUNet_raw_data_base'] = intermediate_dir
os.environ['nnUNet_preprocessed'] = output_dir
os.environ['RESULTS_FOLDER'] = pretrained_model_dir_new

## Download the pretrained model

In [ ]:
# !nnUNet_print_available_pretrained_models
# !nnUNet_download_pretrained_model Task001_BrainTumour

# 4. Prepare the data as per nnUnet's requirement

In [ ]:
# path_ts = '/kaggle/working/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs'

In [ ]:
# for fn in Path("./t1w_resample").ls(): 
#     pat_id = fn.name.split('.')[0].split('_')[0]
#     shutil.copy(f'/kaggle/working/t1w_resample/{pat_id}_0001.nii.gz', f'{path_ts}')
#     shutil.copy(f'/kaggle/working/t1wce_resample/{pat_id}_0002.nii.gz', f'{path_ts}/{pat_id}_0002.nii.gz')
#     shutil.copy(f'/kaggle/working/t2w_resample/{pat_id}_0003.nii.gz', f'{path_ts}/{pat_id}_0003.nii.gz')
#     shutil.copy(f'/kaggle/working/flair_resample/{pat_id}_0000.nii.gz', f'{path_ts}/{pat_id}_0000.nii.gz')
    
#     os.remove(f'/kaggle/working/t1w_resample/{pat_id}_0001.nii.gz')
#     os.remove(f'/kaggle/working/t1wce_resample/{pat_id}_0002.nii.gz')
#     os.remove(f'/kaggle/working/t2w_resample/{pat_id}_0003.nii.gz')
#     os.remove(f'/kaggle/working/flair_resample/{pat_id}_0000.nii.gz')

# 5. Inference - Make Masks

In [ ]:
# result_dir = os.path.join(main_dir,'nnUNet_Prediction_Results',task_name)
# make_if_dont_exist(result_dir)

# #location where you want save your results, will be created if dont exist
# os.chdir(main_dir)
# !nnUNet_predict -i nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs -o nnUNet_Prediction_Results/Task001_BrainTumour -t 001 -tr nnUNetTrainerV2 -m 3d_fullres --disable_tta 
# os.chdir(base_dir)

## Finally let's check the masks

In [ ]:
# import os
# from fastai.vision.all import *
# import SimpleITK as sitk
# import shutil

In [ ]:
# path_ts = '/kaggle/working/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs'
# path_lbl = '/kaggle/working/nnUNet/nnunet/nnUNet_Prediction_Results/Task001_BrainTumour'
# samp_pat = '00094'

In [ ]:
# def get_array(fn):
#     "opens .nii file and return the array"
#     img = sitk.ReadImage(str(fn))
#     imgd = sitk.GetArrayFromImage(img)
#     return imgd

# def plot_slice(imgd, sli):
#     "given an image of shape slices x height x width, plots a slice"
#     plt.imshow(imgd[sli], cmap='gray')
#     plt.axis('off')
    
# def get_array_plot(fn, sli):
#     imgd = get_array(fn)
#     plot_slice(imgd, sli)

In [ ]:
# get_array_plot(f'{path_ts}/{samp_pat}_0003.nii.gz', 85)

In [ ]:
# get_array_plot(f'{path_lbl}/{samp_pat}.nii.gz', 100)

In [ ]:
# print_img_dims(f'{path_lbl}/{samp_pat}.nii.gz')

# Generator of segmented images

In [ ]:
import glob
# print(glob.glob("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*"))
paths = glob.glob("../input/rsna-miccai-brain-tumor-radiogenomic-classification/train/*")
samples = [Path(path) for path in paths]

In [ ]:
def batch_inference(paths, batch_size):
    """
    paths: List of Path objects pointing to the directories with individual patients data (patient_dir).
    This directories should follow contain the following subdirs:
    patient_dir
    |----FLAIR
         |----DICOM slices
    |----T1w
         |----DICOM slices
    |----T1wCE
         |----DICOM slices
    |----T2w
         |----DICOM slices
    """
    
    def batch(paths_by_mri_types, n):
        assert len(paths_by_mri_types) == 4
        assert len(paths_by_mri_types[0]) == len(paths_by_mri_types[1]) == len(paths_by_mri_types[2]) == len(paths_by_mri_types[3]), "There are different number of patients for different MRI types."
        l = len(paths_by_mri_types[0])
        for ndx in range(0, l, n):
            t2w_paths_batch = paths_by_mri_types[0][ndx:min(ndx + n, l)]
            t1wce_paths_batch = paths_by_mri_types[1][ndx:min(ndx + n, l)]
            t1w_paths_batch = paths_by_mri_types[2][ndx:min(ndx + n, l)]
            flair_paths_batch = paths_by_mri_types[3][ndx:min(ndx + n, l)]
            yield t2w_paths_batch, t1wce_paths_batch, t1w_paths_batch, flair_paths_batch
            
    def mri_type_to_path_suffix(mri_type):
        if mri_type == "FLAIR":
            return "0000"
        if mri_type == "T2w":
            return "0003"
        if mri_type == "T1wCE":
            return "0002"
        if mri_type == "T1w":
            return "0001"
        else:
            raise NotImplementedError
            
    def get_paths_per_mri_type(paths):
        path_train_t2w, path_train_t1wce, path_train_t1w, path_train_flair = [],[],[],[]
        for each in paths:
            path_train_t2w.append(each.ls()[0])
            path_train_t1wce.append(each.ls()[1])
            path_train_t1w.append(each.ls()[2])
            path_train_flair.append(each.ls()[3])
        return path_train_t2w, path_train_t1wce, path_train_t1w, path_train_flair

    ref_image = sitk.ReadImage('../input/sri24-dataset/sri24/spgr.nii', sitk.sitkFloat32)
#     path_ts = '/kaggle/working/nnUNet/nnunet/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs'
    path_ts = os.path.join(intermediate_dir, "nnUNet_raw_data/Task001_BrainTumour/imagesTs")
    
    i = 0
    for paths_batch_per_mri_type in batch(get_paths_per_mri_type(paths), batch_size):
        print(f"Batch size: {batch_size}")
        print(f"Processing batch: {i}")
#         print(f"Paths batches for types {paths_batch_per_mri_type}")
        i += 1
        for path_batch in paths_batch_per_mri_type:
            mri_type = str(path_batch[0]).split('/')[-1]
            print(f"Processing MRI type: {mri_type}")
#             print(f"Paths for batch {path_batch}")
            for path in path_batch:
                nifti_img = dicom2nifti(path, out_dir=None, save=False)
                pat_id = str(path).split('/')[-2]
                print(f"Processing patient: {pat_id}")
#                 print(f"Path: {str(path)}")
                path_suffix_for_mri_type = mri_type_to_path_suffix(mri_type)
                final_path = f"{path_ts}/{pat_id}_{path_suffix_for_mri_type}.nii.gz"
                normalized_img = resample_nifti(image_dir=None, ref_image=ref_image, fn=final_path, nifti_img=nifti_img, save=True)
            
#         os.chdir(nnunet_dir)
        !nnUNet_predict -i input/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs -o output/segmented -t 001 -tr nnUNetTrainerV2 -m 3d_fullres --disable_tta 
        os.chdir(base_dir)
        
        !rm -rf /kaggle/working/input/nnUNet_raw_data_base/nnUNet_raw_data/Task001_BrainTumour/imagesTs/*
        
        

In [ ]:
batch_inference(samples, batch_size=20)

# Check the segmented output

In [ ]:
def get_array(fn):
    "opens .nii file and return the array"
    img = sitk.ReadImage(str(fn))
    imgd = sitk.GetArrayFromImage(img)
    return imgd

def plot_slice(imgd, sli):
    "given an image of shape slices x height x width, plots a slice"
    plt.imshow(imgd[sli], cmap='gray')
    plt.axis('off')
    
def get_array_plot(fn, sli):
    imgd = get_array(fn)
    plot_slice(imgd, sli)

In [ ]:
path = '/kaggle/working/output/segmented/00688.nii.gz'
get_array_plot(path, 50)